In [1]:
#Libraries
#Dataframe and arrays
import pandas as pd
import numpy as np

#Graphing/ Charting/ Visualisations
import matplotlib as plt

In [2]:
df = pd.read_excel('../data/online_fashion.xlsx')

In [3]:
#Find & Remove Strange StockCodes
df_unique_stock_codes = df['StockCode'].unique()
df_unique_stock_codes = pd.DataFrame(df_unique_stock_codes)
df_unique_stock_codes['len'] = df_unique_stock_codes[0].astype(str).str.len()
df_unique_stock_codes.rename(columns={0:'StockCode'}, inplace=True)
df_unique_stock_codes.sort_values(by='len')
df_unique_stock_codes['len'] = df_unique_stock_codes['len'].astype(int)
df_stock_to_drop_stg = df_unique_stock_codes.loc[(df_unique_stock_codes['len'] >= 8) | (df_unique_stock_codes['len'] <= 4)].sort_values(by='len')
df_stock_to_drop_stg.reset_index(drop=True)
df_stock_to_drop = pd.concat([df_stock_to_drop_stg.iloc[0:10], df_stock_to_drop_stg.iloc[-6:], df_stock_to_drop_stg.iloc[-9:-8] ])
df_stock_to_drop

df = df[~(df['StockCode'].isin(df_stock_to_drop.StockCode) )]

df.shape

(538959, 8)

In [35]:
df['Month'] = df['InvoiceDate'].dt.month_name()

In [21]:
#Identify if Return(1) or Sale(0)
def transtype(row): 
    """
    Input:
    Output:
    
    The purpose of this function is to .....
    """
    if row['Quantity'] <= 0 : 
        return 1
    else: 
        return 0

df['Return'] = df.apply(transtype, axis=1)
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Return
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,0
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0


In [25]:
df['Rev'] = (df.Quantity * df.UnitPrice)
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Return,Rev
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,0,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,0,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0,20.34


In [10]:
#Number of rows with no CustomerID
len(df[df['CustomerID'].isnull()]) #134,050

134050

In [15]:
#Number of items with no CustomerID
df[df['CustomerID'].isnull()].Quantity.sum() #264,390

264390

In [22]:
#Number of items sold with no CustomerID
df[(df['CustomerID'].isnull() &  (df.Return == 0) )].Quantity.sum() #47,3893

473893

In [23]:
#Number of items returned with no CustomerID
df[(df['CustomerID'].isnull() &  (df.Return == 1) )].Quantity.sum() #-209,503

-209503

In [26]:
#Value of items sold with no CustomerID
df[(df['CustomerID'].isnull() &  (df.Return == 0) )].Rev.sum() #1,510,366.41

1510366.41

In [27]:
#Value of items returned with no CustomerID
df[(df['CustomerID'].isnull() &  (df.Return == 1) )].Rev.sum() -4,320.85

-4320.85

In [39]:
df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
Return                  int64
Rev                   float64
Month                  object
dtype: object

In [40]:
#Number of items returned with no CustomerID in May
df[(df['CustomerID'].isnull() &  (df.Return == 1) & (df.Month == 'May') )].Quantity.sum() #13,172

-13172

In [42]:
#Vale of items returned with no CustomerID in May
df[(df['CustomerID'].isnull() &  (df.Return == 1) & (df.Month == 'May') )].Rev.sum() #-19.98

-19.98

In [41]:
#Number of items returned with no CustomerID in June
df[(df['CustomerID'].isnull() &  (df.Return == 1) & (df.Month == 'June') )].Quantity.sum() #45,917

-45917

In [43]:
#Number of items returned with no CustomerID in June
df[(df['CustomerID'].isnull() &  (df.Return == 1) & (df.Month == 'June') )].Rev.sum() #-1,108.13

-1108.13